## THE BATTLE OF NEIGHBORHOODS

### Problem: Suggesting nearby food, coffee, nightlife, fun and shopping places to tourists visiting Kolkata, IN based on the location of their hotel.


#### Introduction: Kolkata is a very busy tourist destination as it attracts tourists from all corners of the world. The project looks to suggest nearby places where the tourists can hangout without having to spend much time on looking for places to hang out at.

#### Data: All the data used in this notebook are fetched using FOURSQUARE API.

#### Methodology: The process used is K-means Clustering.

Firstly, we will import all the primary libraries needed.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
!pip install geocoder
import geocoder
import os
!pip install folium
import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

print('Libraries imported.')

Libraries imported.


Now, we import the function 'KMeans' that will perform k-means clustering from the 'cluster' library from 'sklearn' module.

In [2]:
from sklearn.cluster import KMeans

In [66]:
CLIENT_ID = "2C2VEMD5PZD4UOAPK0RYGRBB4XA3PCIMABOKFVMUVO21FEAC"
CLIENT_SECRET = "C2SK0MD0BWFG3DMDONNMEJ55HEWGIJBRQQWBFED45HDAAWCR"

In [4]:
VERSION = '20180604'
RADIUS = 20000
LIMIT = 50

Now, we will import:

1. json_normalize: to normalize semi-structured JSON data into a flat table.

2. Nominatim: to retrieve the geo-coordinates (latitude, longitude) of a location.

In [5]:
from pandas.io.json import json_normalize
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - geopy


The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0
  python_abi         conda-forge/win-64::python_abi-3.7-1_cp37m

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.2-py37_0 --> conda-forge::conda-4.8.3-py37hc8dfbb8_1


Preparing transaction: ...working... done
Verifying transaction: ...working... failed




==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




Now, we will retrieve the geo-coordinates of Kolkata, IN.

In [7]:
address = 'Kolkata, IN'

geolocator = Nominatim(user_agent="foursquare_explorer")
location = geolocator.geocode(address)
kol_latitude = location.latitude
kol_longitude = location.longitude
print('The geograpical coordinates of Kolkata are ({}, {}).'.format(kol_latitude, kol_longitude))

The geograpical coordinates of Kolkata are (22.54541245, 88.3567751581234).


Now, we will retrieve a list of hotels using Foursquare API, where the tourists can put up and process the data in the form of a table and create a dataframe of the same.

In [8]:
search_query = 'suite hotel'

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, kol_latitude, kol_longitude, VERSION, search_query, RADIUS, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YSEKPXDXT1RAGTAL1IG51MZCFVJJPC2AMVX3QOZW4DMKGIKA&client_secret=GV53UDJBDHPWTCCW4CLGOBRJYMTFHVNAMWKWTM42ASFH1ISA&ll=22.54541245,88.3567751581234&v=20180604&query=suite hotel&radius=20000&limit=50'

In [9]:
hotel_results = requests.get(url).json()
hotel_results

{'meta': {'code': 200, 'requestId': '5ee634970ed92b6704b5de5d'},
 'response': {'venues': [{'id': '56d53dfb498e4f009bdf144b',
    'name': 'Swiss Suites Hotel',
    'location': {'lat': 22.530879,
     'lng': 88.354877,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.530879,
       'lng': 88.354877}],
     'distance': 1629,
     'cc': 'IN',
     'country': 'India',
     'formattedAddress': ['India']},
    'categories': [{'id': '4f4530a74b9074f6e4fb0100',
      'name': 'Boarding House',
      'pluralName': 'Boarding Houses',
      'shortName': 'Boarding',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592145241',
    'hasPerk': False},
   {'id': '4c83e8a577be76b00c7fab76',
    'name': 'Bigg Boss Hotel',
    'location': {'address': '11/1A, Sarojini Naidu Sarani',
     'lat': 22.543060420316177,
     'lng': 88.35768342018127,
     'labeledLatLngs': [{'label': 'display',
  

In [10]:
# assign relevant part of JSON to venues
hotels = hotel_results['response']['venues']

# tranform venues into a dataframe
hotel_df = json_normalize(hotels)
hotel_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet
0,56d53dfb498e4f009bdf144b,Swiss Suites Hotel,"[{'id': '4f4530a74b9074f6e4fb0100', 'name': 'B...",v-1592145241,False,22.530879,88.354877,"[{'label': 'display', 'lat': 22.530879, 'lng':...",1629,IN,India,[India],NaN,NaN,NaN,NaN,NaN
1,4c83e8a577be76b00c7fab76,Bigg Boss Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592145241,False,22.543060,88.357683,"[{'label': 'display', 'lat': 22.54306042031617...",277,IN,India,"[11/1A, Sarojini Naidu Sarani, Kolkata 700017,...","11/1A, Sarojini Naidu Sarani",700017,Kolkata,West Bengal,NaN
2,4bd858e52ecdce7284bbd0f2,Senator Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592145241,False,22.542871,88.352128,"[{'label': 'display', 'lat': 22.54287079889197...",555,IN,India,"[15, Camac St, Kolkata 700017, West Bengal, In...","15, Camac St",700017,Kolkata,West Bengal,NaN
3,53069a7f498ef0769863af83,Shiraz Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592145241,False,22.546671,88.361603,"[{'label': 'display', 'lat': 22.54667091369629...",515,IN,India,"[56, Park St, Kolkata 700017, West Bengal, India]","56, Park St",700017,Kolkata,West Bengal,NaN
4,517f4754e4b04ce3d2d2594e,Hotel Diamond Suites,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1592145241,False,22.543205,88.361984,"[{'label': 'display', 'lat': 22.543205, 'lng':...",589,IN,India,"[17/1, Karaya Rd, Kolkata, West Bengal, India]","17/1, Karaya Rd",NaN,Kolkata,West Bengal,NaN


In [11]:
hotel_df = hotel_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
hotel_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [12]:
hotel_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,Swiss Suites Hotel,56d53dfb498e4f009bdf144b,22.530879,88.354877,['India']
1,Bigg Boss Hotel,4c83e8a577be76b00c7fab76,22.543060,88.357683,"['11/1A, Sarojini Naidu Sarani', 'Kolkata 700017', 'West Bengal', 'India']"
2,Senator Hotel,4bd858e52ecdce7284bbd0f2,22.542871,88.352128,"['15, Camac St', 'Kolkata 700017', 'West Bengal', 'India']"
3,Shiraz Hotel,53069a7f498ef0769863af83,22.546671,88.361603,"['56, Park St', 'Kolkata 700017', 'West Bengal', 'India']"
4,Hotel Diamond Suites,517f4754e4b04ce3d2d2594e,22.543205,88.361984,"['17/1, Karaya Rd', 'Kolkata', 'West Bengal', 'India']"
5,Hotel Victerrace,502f111ee4b0504d9e34ee8c,22.542445,88.352461,"['1B, Gorky Tce', 'Kolkata 700017', 'West Bengal', 'India']"
6,Kenilworth Hotel,4bd1a0be5e0cce72d6bda184,22.546211,88.350133,"['2, Nandalal Bose Sarani', 'Kolkata 700071', 'West Bengal', 'India']"
7,Roland Hotel,4df6ee761838c789e8b80202,22.536342,88.356986,"['28A, Roland Rd', 'Kolkata 700020', 'West Bengal', 'India']"
8,Hotel Hindusthan International,4bc788f793bdeee1917d37ae,22.541075,88.350490,"['235/1, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']"
9,Golden Park Hotel,4bd1079120cd996098a62e9e,22.547619,88.349605,"['13, Ho Chi Minh Sarani', 'Kolkata 700071', 'West Bengal', 'India']"


Now, let's say the tourist is staying in The Park Hotel.

In [16]:
hotel_id = -1
while hotel_id not in range(50):
    hotel_id = int(input("Enter the index of the hotel: "))
user_input = {'hotel_name': hotel_df['Name'][hotel_id], 'hotel_id': hotel_df['ID'][hotel_id], 'latitude': hotel_df['Latitude'][hotel_id], 'longitude': hotel_df['Longitude'][hotel_id], 'Address': hotel_df['Address'][hotel_id]} 

Enter the index of the hotel: 10


In [17]:
print('Details of user input:')
print('Name:', user_input['hotel_name'])
print('ID:', user_input['hotel_id'])
print('Latitude:', user_input['latitude'])
print('Longitude:', user_input['longitude'])
print('Address:', user_input['Address'])

Details of user input:
Name: The Park Hotel
ID: 4d1348110ad2f04d513fb254
Latitude: 22.55388349859243
Longitude: 88.3516722970781
Address: ['17, Park St', 'Kolkata 700016', 'West Bengal', 'India']


In [31]:
import math

### Now we will retrieve data about places for food nearby using the Foursquare API and then perform K-Means Clustering and return the suggestions of nearby places to visit.

In [18]:
food_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, user_input['latitude'], user_input['longitude'], VERSION, 'food', RADIUS, LIMIT)
food_url

'https://api.foursquare.com/v2/venues/search?client_id=YSEKPXDXT1RAGTAL1IG51MZCFVJJPC2AMVX3QOZW4DMKGIKA&client_secret=GV53UDJBDHPWTCCW4CLGOBRJYMTFHVNAMWKWTM42ASFH1ISA&ll=22.55388349859243,88.3516722970781&v=20180604&query=food&radius=20000&limit=50'

In [19]:
food_results = requests.get(food_url).json()
food_results

{'meta': {'code': 200, 'requestId': '5ee64139be946f5f08461348'},
 'response': {'venues': [{'id': '4fc631bbe4b005fcba5f8871',
    'name': 'Food Bazaar',
    'location': {'address': 'Pantaloons',
     'lat': 22.546905083439395,
     'lng': 88.35339188575745,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.546905083439395,
       'lng': 88.35339188575745}],
     'distance': 796,
     'postalCode': '700016',
     'cc': 'IN',
     'city': 'Kolkata',
     'state': 'West Bengal',
     'country': 'India',
     'formattedAddress': ['Pantaloons',
      'Kolkata 700016',
      'West Bengal',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d120951735',
      'name': 'Food Court',
      'pluralName': 'Food Courts',
      'shortName': 'Food Court',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_foodcourt_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592148326',
    'hasPerk': False},
   {'id': '5dc6c72c1a7196000716966

In [20]:
# assign relevant part of JSON to venues
food = food_results['response']['venues']

# tranform venues into a dataframe
food_df = json_normalize(food)
food_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4fc631bbe4b005fcba5f8871,Food Bazaar,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1592148326,False,Pantaloons,22.546905,88.353392,"[{'label': 'display', 'lat': 22.54690508343939...",796,700016,IN,Kolkata,West Bengal,India,"[Pantaloons, Kolkata 700016, West Bengal, India]",NaN,NaN
1,5dc6c72c1a71960007169661,The Firangi Food Smith,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1592148326,False,"12B/2, Park St",22.553484,88.351783,"[{'label': 'display', 'lat': 22.55348436152786...",45,700016,IN,Kolkata,West Bengal,India,"[12B/2, Park St, Kolkata 700016, West Bengal, ...",NaN,NaN
2,5b0ab58ee1f228002c5797d7,R C S Fast Food,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1592148326,False,"229, Acharya Jagadish Chandra Bose Rd",22.541059,88.353649,"[{'label': 'display', 'lat': 22.54105877495912...",1442,700020,IN,Kolkata,West Bengal,India,"[229, Acharya Jagadish Chandra Bose Rd, Kolkat...",NaN,NaN
3,598eb4d98d0a5310231f7437,Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1592148326,False,Quest Mall,22.539206,88.365499,"[{'label': 'display', 'lat': 22.53920574076206...",2165,700017,IN,Kolkata,West Bengal,India,"[Quest Mall, Kolkata 700017, West Bengal, India]",NaN,NaN
4,4dd3a4d9d1643acaa1e530b7,Sourav's the food pavilion,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592148326,False,NaN,22.552067,88.352458,"[{'label': 'display', 'lat': 22.5520670413971,...",217,NaN,IN,NaN,NaN,India,[India],NaN,NaN


In [21]:
food_df = food_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
food_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [22]:
food_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,Food Bazaar,4fc631bbe4b005fcba5f8871,22.546905,88.353392,"['Pantaloons', 'Kolkata 700016', 'West Bengal', 'India']"
1,The Firangi Food Smith,5dc6c72c1a71960007169661,22.553484,88.351783,"['12B/2, Park St', 'Kolkata 700016', 'West Bengal', 'India']"
2,R C S Fast Food,5b0ab58ee1f228002c5797d7,22.541059,88.353649,"['229, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']"
3,Food Court,598eb4d98d0a5310231f7437,22.539206,88.365499,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']"
4,Sourav's the food pavilion,4dd3a4d9d1643acaa1e530b7,22.552067,88.352458,['India']
5,Food Court,4fd20db1e4b07915217167a0,22.563115,88.323727,"['Avani Riverside Mall', 'Hāora 711102', 'West Bengal', 'India']"
6,Savitri Food Centre,53f36275498efdaea76db434,22.557699,88.352986,['India']
7,Jullet Fast Food Centre,511e1581e4b01f60b233a167,22.552562,88.356137,['India']
8,Savitri Fast Food Center,5a6c6c5d9b047348f8e253b1,22.558012,88.353485,"['Kolkata 700016', 'West Bengal', 'India']"
9,Aqua Java Fast Food Pvt. Ltd.,4e71c57ce4cd7aa6d6f40a53,22.546865,88.354229,"['10, Wood St', 'Kolkata 700017', 'West Bengal', 'India']"


K-Means cluustering with 8 clusters.

In [23]:
food_clf = KMeans(n_clusters = 8, init = 'k-means++')
food_clf.fit(food_df[['Latitude', 'Longitude']])
food_labels = food_clf.labels_
food_centers = food_clf.cluster_centers_

In [25]:
food_df['Label'] = 0

for i in range(food_df.shape[0]):
    food_df['Label'][i] = food_labels[i]

food_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,ID,Latitude,Longitude,Address,Label
0,Food Bazaar,4fc631bbe4b005fcba5f8871,22.546905,88.353392,"[Pantaloons, Kolkata 700016, West Bengal, India]",5
1,The Firangi Food Smith,5dc6c72c1a71960007169661,22.553484,88.351783,"[12B/2, Park St, Kolkata 700016, West Bengal, ...",0
2,R C S Fast Food,5b0ab58ee1f228002c5797d7,22.541059,88.353649,"[229, Acharya Jagadish Chandra Bose Rd, Kolkat...",5
3,Food Court,598eb4d98d0a5310231f7437,22.539206,88.365499,"[Quest Mall, Kolkata 700017, West Bengal, India]",5
4,Sourav's the food pavilion,4dd3a4d9d1643acaa1e530b7,22.552067,88.352458,[India],0


Let's visualise the clusters.

In [27]:
# create map
map_clusters = folium.Map(location = [kol_latitude, kol_longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(food_df['Latitude'], food_df['Longitude'], food_df['Name'], food_df['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

Now, we will find out the nearest cluster.

In [28]:
food_centers

array([[22.56236458, 88.35156332],
       [22.50192697, 88.36316374],
       [22.57845396, 88.41483432],
       [22.53177299, 88.34780705],
       [22.62019081, 88.39625359],
       [22.54229635, 88.36029102],
       [22.57816377, 88.47553968],
       [22.51344896, 88.39960098]])

Let's find out the respective distances of each cluster center from the hotel.

In [39]:
import math

dist = []

for i in range(8):
    dist.append(math.sqrt((kol_latitude - food_centers[i][0])**2 + (kol_longitude - food_centers[i][1])**2))
    
dist

[0.017735213197673683,
 0.04395225480523355,
 0.0668027516008359,
 0.01632365906499604,
 0.08455973687721817,
 0.004698018610840459,
 0.12319765166440198,
 0.053438896830498066]

Let's find out the nearest cluster.

In [40]:
idx = dist.index(min(dist))
idx

5

Now, we will return the nearest cluster places.

In [44]:
food_df[food_df['Label'] == idx].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Food Bazaar,4fc631bbe4b005fcba5f8871,22.546905,88.353392,"['Pantaloons', 'Kolkata 700016', 'West Bengal', 'India']",5
2,R C S Fast Food,5b0ab58ee1f228002c5797d7,22.541059,88.353649,"['229, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",5
3,Food Court,598eb4d98d0a5310231f7437,22.539206,88.365499,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']",5
9,Aqua Java Fast Food Pvt. Ltd.,4e71c57ce4cd7aa6d6f40a53,22.546865,88.354229,"['10, Wood St', 'Kolkata 700017', 'West Bengal', 'India']",5
13,Aqua Java Fast Food Pvt. Ltd.,4bbe3eda79eaef3b07e4aec3,22.542949,88.355867,"['8/2, Loudon St', 'Kolkata 700016', 'West Bengal', 'India']",5
14,Hunger Food Court,5ca04aaf18d43b002c8c4b59,22.544605,88.354576,"['700017', 'West Bengal', 'India']",5
16,Arsh Fast Food,51dac3e2498e0188907115ec,22.550649,88.362094,['India'],5
17,City Fast Food Center,5916c41f6bd36b0dc9458694,22.551789,88.363664,"['Acharya Jagadish Chandra Bose Rd (Beniapukur Road)', 'Kolkata 700017', 'West Bengal', 'India']",5
19,Al falah fast food,5088fd48e4b0b15cd5f580ad,22.543978,88.359619,"['Kolkata', 'West Bengal', 'India']",5
21,Shimnan Fast Food Centre,50c3145ce4b0e121076eabb6,22.541098,88.353767,"['Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",5


### Now we will retrieve data about places for coffee nearby using the Foursquare API and then perform K-Means Clustering and return the suggestions of nearby places to visit.

In [45]:
coffee_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, user_input['latitude'], user_input['longitude'], VERSION, 'coffee', RADIUS, LIMIT)
coffee_url

'https://api.foursquare.com/v2/venues/search?client_id=YSEKPXDXT1RAGTAL1IG51MZCFVJJPC2AMVX3QOZW4DMKGIKA&client_secret=GV53UDJBDHPWTCCW4CLGOBRJYMTFHVNAMWKWTM42ASFH1ISA&ll=22.55388349859243,88.3516722970781&v=20180604&query=coffee&radius=20000&limit=50'

In [48]:
coffee_results = requests.get(coffee_url).json()
coffee_results

{'meta': {'code': 200, 'requestId': '5ee646a1c1ce500e4ec3c792'},
 'response': {'venues': [{'id': '4c6e9eae4ee776b0b4b0581b',
    'name': 'Cafe Coffee Day',
    'location': {'address': '4/1, Camac St',
     'lat': 22.54547821065463,
     'lng': 88.3525013923645,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.54547821065463,
       'lng': 88.3525013923645}],
     'distance': 939,
     'postalCode': '700016',
     'cc': 'IN',
     'city': 'Kolkata',
     'state': 'West Bengal',
     'country': 'India',
     'formattedAddress': ['4/1, Camac St',
      'Kolkata 700016',
      'West Bengal',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592149953',
    'hasPerk': False},
   {'id': '4c6ea5834ee776b055e1581b',
    'name': 'Cafe C

In [49]:
# assign relevant part of JSON to venues
coffee = coffee_results['response']['venues']

# tranform venues into a dataframe
coffee_df = json_normalize(coffee)
coffee_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4c6e9eae4ee776b0b4b0581b,Cafe Coffee Day,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1592149953,False,"4/1, Camac St",22.545478,88.352501,"[{'label': 'display', 'lat': 22.54547821065463...",939,700016,IN,Kolkata,West Bengal,India,"[4/1, Camac St, Kolkata 700016, West Bengal, I...",NaN,NaN
1,4c6ea5834ee776b055e1581b,Cafe Coffee Day,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1592149953,False,Park St,22.552775,88.352717,"[{'label': 'display', 'lat': 22.55277532104876...",163,700017,IN,Kolkata,West Bengal,India,"[Park St, Kolkata 700017, West Bengal, India]",NaN,NaN
2,5167a710e4b08b494a5adeed,Cafe Coffee Day,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1592149953,False,Park St,22.555407,88.350313,"[{'label': 'display', 'lat': 22.55540656126555...",219,700016,IN,Kolkata,West Bengal,India,"[Park St, Kolkata 700016, West Bengal, India]",NaN,NaN
3,4cbdd127035d236aeb1aed4e,Café Coffee Day,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1592149953,False,Big Bazaar,22.565919,88.369635,"[{'label': 'display', 'lat': 22.56591873648823...",2281,700106,IN,Kolkata,West Bengal,India,"[Big Bazaar, Kolkata 700106, West Bengal, India]",NaN,NaN
4,52c16ed711d2e75296b8c7f6,The Coffee Bean & Tea Leaf,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1592149953,False,Quest Mall,22.538809,88.365524,"[{'label': 'display', 'lat': 22.53880898011972...",2200,700017,IN,Kolkata,West Bengal,India,"[Quest Mall, Kolkata 700017, West Bengal, India]",NaN,NaN


In [50]:
coffee_df = coffee_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
coffee_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [51]:
coffee_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,Cafe Coffee Day,4c6e9eae4ee776b0b4b0581b,22.545478,88.352501,"['4/1, Camac St', 'Kolkata 700016', 'West Bengal', 'India']"
1,Cafe Coffee Day,4c6ea5834ee776b055e1581b,22.552775,88.352717,"['Park St', 'Kolkata 700017', 'West Bengal', 'India']"
2,Cafe Coffee Day,5167a710e4b08b494a5adeed,22.555407,88.350313,"['Park St', 'Kolkata 700016', 'West Bengal', 'India']"
3,Café Coffee Day,4cbdd127035d236aeb1aed4e,22.565919,88.369635,"['Big Bazaar', 'Kolkata 700106', 'West Bengal', 'India']"
4,The Coffee Bean & Tea Leaf,52c16ed711d2e75296b8c7f6,22.538809,88.365524,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']"
5,Indian Coffee House,4c2db6124e132d7fb6d79cdf,22.576187,88.364013,"['15, Bankim Chatterjee St', 'Kolkata 700073', 'West Bengal', 'India']"
6,Cafe Coffee Day,4c6eaf425d089c7487bf07c3,22.547381,88.354433,"['Wood St', 'Kolkata 700016', 'West Bengal', 'India']"
7,Cafe Coffee Day Kolkata Office,50347ce4e4b0bf8f2d057132,22.545424,88.352783,"['Camac St', 'Kolkata 700017', 'West Bengal', 'India']"
8,Cafe Coffee Day,4c25e307136d20a1414fe461,22.516449,88.366905,"['Ballygunge Gardens', 'Kolkata 700019', 'West Bengal', 'India']"
9,Le Coffee Creme,50d6f7f4e4b0252165c01b08,22.587859,88.409336,"['EC 26, Sector 1, Salt Lake', 'Kolkata 700064', 'West Bengal', 'India']"


K-Means Clustering with 8 clusters.

In [52]:
coffee_clf = KMeans(n_clusters = 8, init = 'k-means++')
coffee_clf.fit(coffee_df[['Latitude', 'Longitude']])
coffee_labels = coffee_clf.labels_
coffee_centers = coffee_clf.cluster_centers_

In [53]:
coffee_df['Label'] = 0

for i in range(coffee_df.shape[0]):
    coffee_df['Label'][i] = coffee_labels[i]

coffee_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,ID,Latitude,Longitude,Address,Label
0,Cafe Coffee Day,4c6e9eae4ee776b0b4b0581b,22.545478,88.352501,"[4/1, Camac St, Kolkata 700016, West Bengal, I...",1
1,Cafe Coffee Day,4c6ea5834ee776b055e1581b,22.552775,88.352717,"[Park St, Kolkata 700017, West Bengal, India]",1
2,Cafe Coffee Day,5167a710e4b08b494a5adeed,22.555407,88.350313,"[Park St, Kolkata 700016, West Bengal, India]",1
3,Café Coffee Day,4cbdd127035d236aeb1aed4e,22.565919,88.369635,"[Big Bazaar, Kolkata 700106, West Bengal, India]",6
4,The Coffee Bean & Tea Leaf,52c16ed711d2e75296b8c7f6,22.538809,88.365524,"[Quest Mall, Kolkata 700017, West Bengal, India]",1


Let's visualise the clusters.

In [54]:
# create map
map_clusters = folium.Map(location = [kol_latitude, kol_longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(coffee_df['Latitude'], coffee_df['Longitude'], coffee_df['Name'], coffee_df['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

Now, we will find out the nearest cluster.

In [55]:
coffee_centers

array([[22.58279742, 88.4104073 ],
       [22.54335093, 88.35470746],
       [22.51866373, 88.35771457],
       [22.58533074, 88.28425125],
       [22.49430926, 88.30398657],
       [22.50446916, 88.39701339],
       [22.574917  , 88.36027594],
       [22.49518516, 88.36431548]])

Let's find out the respective distances of each cluster center from the hotel.

In [56]:
import math

dist = []

for i in range(8):
    dist.append(math.sqrt((kol_latitude - coffee_centers[i][0])**2 + (kol_longitude - coffee_centers[i][1])**2))
    
dist

[0.0653761606196351,
 0.0029197992788676094,
 0.026765207957688376,
 0.0827839753101103,
 0.07347224708986652,
 0.05740617267986573,
 0.02971151627011081,
 0.05079012904566458]

Let's find out the nearest cluster.

In [57]:
idx = dist.index(min(dist))
idx

1

Now, we will return the nearest cluster places.

In [59]:
coffee_df[coffee_df['Label'] == idx].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Cafe Coffee Day,4c6e9eae4ee776b0b4b0581b,22.545478,88.352501,"['4/1, Camac St', 'Kolkata 700016', 'West Bengal', 'India']",1
1,Cafe Coffee Day,4c6ea5834ee776b055e1581b,22.552775,88.352717,"['Park St', 'Kolkata 700017', 'West Bengal', 'India']",1
2,Cafe Coffee Day,5167a710e4b08b494a5adeed,22.555407,88.350313,"['Park St', 'Kolkata 700016', 'West Bengal', 'India']",1
4,The Coffee Bean & Tea Leaf,52c16ed711d2e75296b8c7f6,22.538809,88.365524,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']",1
6,Cafe Coffee Day,4c6eaf425d089c7487bf07c3,22.547381,88.354433,"['Wood St', 'Kolkata 700016', 'West Bengal', 'India']",1
7,Cafe Coffee Day Kolkata Office,50347ce4e4b0bf8f2d057132,22.545424,88.352783,"['Camac St', 'Kolkata 700017', 'West Bengal', 'India']",1
12,Coffee World,4d0e13a8801054816a042296,22.534602,88.364883,"['1, Ballygunge Park Rd', 'Kolkata 700019', 'West Bengal', 'India']",1
13,Cafe Coffee Day,4e5b9da345dd705592b256ab,22.543596,88.358161,"['42/A, Rawdon St', 'Kolkata 700017', 'West Bengal', 'India']",1
17,Cafe Coffee Day,5cf94dea8f90d9002347d428,22.541287,88.349830,"['235/2A, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",1
19,Cafe Coffee Day,4c6ea0814ee776b0f4be581b,22.538542,88.351954,"['Woodburn Pk', 'Kolkata 700020', 'West Bengal', 'India']",1


### Now we will retrieve data about places for nightlife nearby using the Foursquare API and then perform K-Means Clustering and return the suggestions of nearby places to visit.

In [89]:
nightlife_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, user_input['latitude'], user_input['longitude'], VERSION, 'pub', RADIUS, LIMIT)
nightlife_url

'https://api.foursquare.com/v2/venues/search?client_id=2C2VEMD5PZD4UOAPK0RYGRBB4XA3PCIMABOKFVMUVO21FEAC&client_secret=C2SK0MD0BWFG3DMDONNMEJ55HEWGIJBRQQWBFED45HDAAWCR&ll=22.55388349859243,88.3516722970781&v=20180604&query=pub&radius=20000&limit=50'

In [90]:
nightlife_results = requests.get(nightlife_url).json()
nightlife_results

{'meta': {'code': 200, 'requestId': '5ee64f8f0ed92b670408d808'},
 'response': {'venues': [{'id': '4feef749e4b0da11fdd6ac72',
    'name': "Sam's Pub",
    'location': {'lat': 22.55331195527922,
     'lng': 88.35167821299912,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.55331195527922,
       'lng': 88.35167821299912}],
     'distance': 63,
     'cc': 'IN',
     'country': 'India',
     'formattedAddress': ['India']},
    'categories': [{'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'shortName': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592152144',
    'hasPerk': False},
   {'id': '4f637bdfe4b08383b0adb6c8',
    'name': "Floriana (Sam's Pub)",
    'location': {'address': '13, Anandilal Poddar Sarani',
     'lat': 22.55318711175468,
     'lng': 88.35095643997192,
     'labeledLatLngs': [{'label': 'display',
    

In [71]:
# assign relevant part of JSON to venues
nightlife = nightlife_results['response']['venues']

# tranform venues into a dataframe
nightlife_df = json_normalize(nightlife)
nightlife_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet
0,4feef749e4b0da11fdd6ac72,Sam's Pub,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1592151787,False,22.553312,88.351678,"[{'label': 'display', 'lat': 22.55331195527922...",63,IN,India,[India],NaN,NaN,NaN,NaN,NaN
1,4f637bdfe4b08383b0adb6c8,Floriana (Sam's Pub),"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1592151787,False,22.553187,88.350956,"[{'label': 'display', 'lat': 22.55318711175468...",106,IN,India,"[13, Anandilal Poddar Sarani, Kolkata 700071, ...","13, Anandilal Poddar Sarani",700071,Kolkata,West Bengal,NaN
2,5260210d11d20a56345c1143,Super Pub,[],v-1592151787,False,22.555179,88.354115,"[{'label': 'display', 'lat': 22.5551789067686,...",289,IN,India,"[Sudder Street, Kolkata, West Bengal, India]",Sudder Street,NaN,Kolkata,West Bengal,NaN
3,5156fb93e4b07cf8c6a2e7b6,Sams Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1592151787,False,22.558224,88.349862,"[{'label': 'display', 'lat': 22.55822380992560...",517,IN,India,[India],NaN,NaN,NaN,NaN,NaN
4,4d7c82a186cfa1439d8ccba0,Olypub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1592151787,False,22.553551,88.351925,"[{'label': 'display', 'lat': 22.55355099631339...",45,IN,India,"[21, Park St, Kolkata 700016, West Bengal, India]","21, Park St",700016,Kolkata,West Bengal,NaN


In [73]:
nightlife_df = nightlife_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
nightlife_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [74]:
nightlife_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,Sam's Pub,4feef749e4b0da11fdd6ac72,22.553312,88.351678,['India']
1,Floriana (Sam's Pub),4f637bdfe4b08383b0adb6c8,22.553187,88.350956,"['13, Anandilal Poddar Sarani', 'Kolkata 700071', 'West Bengal', 'India']"
2,Super Pub,5260210d11d20a56345c1143,22.555179,88.354115,"['Sudder Street', 'Kolkata', 'West Bengal', 'India']"
3,Sams Pub,5156fb93e4b07cf8c6a2e7b6,22.558224,88.349862,['India']
4,Olypub,4d7c82a186cfa1439d8ccba0,22.553551,88.351925,"['21, Park St', 'Kolkata 700016', 'West Bengal', 'India']"
5,London Pub,583d8075ea29b85601cdf158,22.547619,88.349700,"['Golden Parkk, Chowringhee', 'India']"
6,Mags Pubs,4bb4a547613fb7133dfa93e6,22.553347,88.352034,"['12K, Park St', 'Kolkata 700016', 'West Bengal', 'India']"
7,Rishirich Soda Pub,52dd1ccc498e9f6c1d09befe,22.540038,88.361610,"['Beck Bagan Row', 'Kolkata 700017', 'West Bengal', 'India']"
8,Soda Pub,4fdb69ede4b08187e9dabda9,22.538066,88.345485,"['Sambhu Nath Pandit St', 'Kolkata 700020', 'West Bengal', 'India']"
9,Rishirich Soda Pub,529e921511d255ec089af691,22.533329,88.347609,"['23A/1/B, Justice Dwarka Nath Rd', 'Kolkata 700020', 'West Bengal', 'India']"


K-Means Clustering with 8 clusters.

In [75]:
nightlife_clf = KMeans(n_clusters = 8, init = 'k-means++')
nightlife_clf.fit(nightlife_df[['Latitude', 'Longitude']])
nightlife_labels = nightlife_clf.labels_
nightlife_centers = nightlife_clf.cluster_centers_

In [77]:
nightlife_df['Label'] = 0

for i in range(nightlife_df.shape[0]):
    nightlife_df['Label'][i] = nightlife_labels[i]

nightlife_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,ID,Latitude,Longitude,Address,Label
0,Sam's Pub,4feef749e4b0da11fdd6ac72,22.553312,88.351678,[India],5
1,Floriana (Sam's Pub),4f637bdfe4b08383b0adb6c8,22.553187,88.350956,"[13, Anandilal Poddar Sarani, Kolkata 700071, ...",5
2,Super Pub,5260210d11d20a56345c1143,22.555179,88.354115,"[Sudder Street, Kolkata, West Bengal, India]",5
3,Sams Pub,5156fb93e4b07cf8c6a2e7b6,22.558224,88.349862,[India],5
4,Olypub,4d7c82a186cfa1439d8ccba0,22.553551,88.351925,"[21, Park St, Kolkata 700016, West Bengal, India]",5


Let's visualise the clusters.

In [78]:
# create map
map_clusters = folium.Map(location = [kol_latitude, kol_longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(nightlife_df['Latitude'], nightlife_df['Longitude'], nightlife_df['Name'], nightlife_df['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

Now, we will find out the nearest cluster.

In [79]:
nightlife_centers

array([[22.53595205, 88.35001664],
       [22.367823  , 88.27141   ],
       [22.61621874, 88.40129232],
       [22.62452013, 88.23702073],
       [22.51999327, 88.39191955],
       [22.56140433, 88.35430562],
       [22.50970107, 88.35908409],
       [22.58036795, 88.47208518]])

Let's find out the respective distances of each cluster center from the hotel.

In [80]:
import math

dist = []

for i in range(8):
    dist.append(math.sqrt((kol_latitude - nightlife_centers[i][0])**2 + (kol_longitude - nightlife_centers[i][1])**2))
    
dist

[0.011626554433375508,
 0.19704117075559216,
 0.08363796238915347,
 0.1435240346662241,
 0.04337352910748768,
 0.01618143523493895,
 0.03578594057742184,
 0.1204918548642128]

Let's find out the nearest cluster.

In [81]:
idx = dist.index(min(dist))
idx

0

In [91]:
nightlife_df[nightlife_df['Label'] == idx].head(4).style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
5,London Pub,583d8075ea29b85601cdf158,22.547619,88.349700,"['Golden Parkk, Chowringhee', 'India']",0
7,Rishirich Soda Pub,52dd1ccc498e9f6c1d09befe,22.540038,88.361610,"['Beck Bagan Row', 'Kolkata 700017', 'West Bengal', 'India']",0
8,Soda Pub,4fdb69ede4b08187e9dabda9,22.538066,88.345485,"['Sambhu Nath Pandit St', 'Kolkata 700020', 'West Bengal', 'India']",0
9,Rishirich Soda Pub,529e921511d255ec089af691,22.533329,88.347609,"['23A/1/B, Justice Dwarka Nath Rd', 'Kolkata 700020', 'West Bengal', 'India']",0


### Now we will retrieve data about places for fun nearby using the Foursquare API and then perform K-Means Clustering and return the suggestions of nearby places to visit.

In [92]:
fun_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, user_input['latitude'], user_input['longitude'], VERSION, 'fun', RADIUS, LIMIT)
fun_url

'https://api.foursquare.com/v2/venues/search?client_id=2C2VEMD5PZD4UOAPK0RYGRBB4XA3PCIMABOKFVMUVO21FEAC&client_secret=C2SK0MD0BWFG3DMDONNMEJ55HEWGIJBRQQWBFED45HDAAWCR&ll=22.55388349859243,88.3516722970781&v=20180604&query=fun&radius=20000&limit=50'

In [94]:
fun_results = requests.get(fun_url).json()
fun_results

{'meta': {'code': 200, 'requestId': '5ee64f89e5fd9920d428a397'},
 'response': {'venues': [{'id': '5446e0e6498ee9a7ef4d7957',
    'name': 'Kolkata Fun Club',
    'location': {'address': '25, Jawaharlal Nehru Rd',
     'lat': 22.54355587260624,
     'lng': 88.34887504577637,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.54355587260624,
       'lng': 88.34887504577637}],
     'distance': 1185,
     'postalCode': '700016',
     'cc': 'IN',
     'city': 'Kolkata',
     'state': 'West Bengal',
     'country': 'India',
     'formattedAddress': ['25, Jawaharlal Nehru Rd',
      'Kolkata 700016',
      'West Bengal',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d11a941735',
      'name': 'Other Nightlife',
      'pluralName': 'Other Nightlife',
      'shortName': 'Nightlife',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592152307',
    'hasPerk': False},

In [95]:
# assign relevant part of JSON to venues
fun = fun_results['response']['venues']

# tranform venues into a dataframe
fun_df = json_normalize(fun)
fun_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5446e0e6498ee9a7ef4d7957,Kolkata Fun Club,"[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1592152307,False,"25, Jawaharlal Nehru Rd",22.543556,88.348875,"[{'label': 'display', 'lat': 22.54355587260624...",1185,700016,IN,Kolkata,West Bengal,India,"[25, Jawaharlal Nehru Rd, Kolkata 700016, West..."
1,52e3b5af11d227b7730a1eaa,Fun In,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1592152307,False,"12/2, Hindustan Rd",22.520317,88.362179,"[{'label': 'display', 'lat': 22.52031724650700...",3889,700029,IN,Kolkata,West Bengal,India,"[12/2, Hindustan Rd, Kolkata 700029, West Beng..."
2,4ffab423e4b0ae40ee877f91,COLORZ - The Funked-Up Store,"[{'id': '4bf58dd8d48988d102951735', 'name': 'A...",v-1592152307,False,"1/A, Grant St",22.564408,88.354348,"[{'label': 'display', 'lat': 22.56440790614458...",1203,700013,IN,Kolkata,West Bengal,India,"[1/A, Grant St, Kolkata 700013, West Bengal, I..."
3,53073e7811d21b85309290fd,Birla Sun Life Mutual Fund,"[{'id': '503287a291d4c4b30a586d65', 'name': 'F...",v-1592152307,False,"3, Camac St",22.542862,88.351836,"[{'label': 'display', 'lat': 22.54286223890219...",1226,700017,IN,Kolkata,West Bengal,India,"[3, Camac St, Kolkata 700017, West Bengal, India]"
4,549fc3a0498ef7f11ab5872d,Funda & Yener Karakaş,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",v-1592152307,False,Bbsnsbbaba,22.532181,88.347939,"[{'label': 'display', 'lat': 22.53218078613281...",2446,NaN,IN,NaN,NaN,India,"[Bbsnsbbaba, India]"


In [96]:
fun_df = fun_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
fun_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [97]:
fun_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,Kolkata Fun Club,5446e0e6498ee9a7ef4d7957,22.543556,88.348875,"['25, Jawaharlal Nehru Rd', 'Kolkata 700016', 'West Bengal', 'India']"
1,Fun In,52e3b5af11d227b7730a1eaa,22.520317,88.362179,"['12/2, Hindustan Rd', 'Kolkata 700029', 'West Bengal', 'India']"
2,COLORZ - The Funked-Up Store,4ffab423e4b0ae40ee877f91,22.564408,88.354348,"['1/A, Grant St', 'Kolkata 700013', 'West Bengal', 'India']"
3,Birla Sun Life Mutual Fund,53073e7811d21b85309290fd,22.542862,88.351836,"['3, Camac St', 'Kolkata 700017', 'West Bengal', 'India']"
4,Funda & Yener Karakaş,549fc3a0498ef7f11ab5872d,22.532181,88.347939,"['Bbsnsbbaba', 'India']"
5,Personalized Funda,51c8fb85498e8a70de1e97c4,22.535519,88.351729,"['9/1, Ram Mohan Dutta Rd', 'Kolkata 700020', 'West Bengal', 'India']"
6,Funtasia Nursery School,5389d552498e450b0a265f8c,22.518536,88.359772,"['186B, Rash Behari Ave', 'Kolkata 700029', 'West Bengal', 'India']"
7,Funskool (India) Limited,5576485a498ebb62a9515758,22.517512,88.354626,"['148, Rash Behari Ave', 'Kolkata 700029', 'West Bengal', 'India']"
8,Fung Fa,52e7cbc2498e9bdd9a1eb232,22.542376,88.392159,"['119/A/1, Matheshwartala Rd', 'Kolkata 700046', 'West Bengal', 'India']"
9,Fung Shway,5768173a498e4c55ffdb76d2,22.514182,88.362286,"['122A, Southern Ave', 'Kolkata 700029', 'West Bengal', 'India']"


K-Means Clustering with 3 clusters.

In [99]:
fun_clf = KMeans(n_clusters = 3, init = 'k-means++')
fun_clf.fit(fun_df[['Latitude', 'Longitude']])
fun_labels = fun_clf.labels_
fun_centers = fun_clf.cluster_centers_

In [100]:
fun_df['Label'] = 0

for i in range(fun_df.shape[0]):
    fun_df['Label'][i] = fun_labels[i]

fun_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,ID,Latitude,Longitude,Address,Label
0,Kolkata Fun Club,5446e0e6498ee9a7ef4d7957,22.543556,88.348875,"[25, Jawaharlal Nehru Rd, Kolkata 700016, West...",0
1,Fun In,52e3b5af11d227b7730a1eaa,22.520317,88.362179,"[12/2, Hindustan Rd, Kolkata 700029, West Beng...",2
2,COLORZ - The Funked-Up Store,4ffab423e4b0ae40ee877f91,22.564408,88.354348,"[1/A, Grant St, Kolkata 700013, West Bengal, I...",0
3,Birla Sun Life Mutual Fund,53073e7811d21b85309290fd,22.542862,88.351836,"[3, Camac St, Kolkata 700017, West Bengal, India]",0
4,Funda & Yener Karakaş,549fc3a0498ef7f11ab5872d,22.532181,88.347939,"[Bbsnsbbaba, India]",0


Let's visualise the clusters.

In [101]:
# create map
map_clusters = folium.Map(location = [kol_latitude, kol_longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(fun_df['Latitude'], fun_df['Longitude'], fun_df['Name'], fun_df['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

Now, we will find out the nearest cluster.

In [102]:
fun_centers

array([[22.54348363, 88.35781418],
       [22.56886121, 88.43371868],
       [22.51498378, 88.35916694]])

Let's find out the respective distances of each cluster center from the hotel.

In [105]:
import math

dist = []

for i in range(3):
    dist.append(math.sqrt((kol_latitude - fun_centers[i][0])**2 + (kol_longitude - fun_centers[i][1])**2))
    
dist

[0.0021908709344954052, 0.08043724311712251, 0.030522528519515098]

Let's find out the nearest cluster.

In [106]:
idx = dist.index(min(dist))
idx

0

In [110]:
fun_df[fun_df['Label'] == idx].drop(3).style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Kolkata Fun Club,5446e0e6498ee9a7ef4d7957,22.543556,88.348875,"['25, Jawaharlal Nehru Rd', 'Kolkata 700016', 'West Bengal', 'India']",0
2,COLORZ - The Funked-Up Store,4ffab423e4b0ae40ee877f91,22.564408,88.354348,"['1/A, Grant St', 'Kolkata 700013', 'West Bengal', 'India']",0
4,Funda & Yener Karakaş,549fc3a0498ef7f11ab5872d,22.532181,88.347939,"['Bbsnsbbaba', 'India']",0
5,Personalized Funda,51c8fb85498e8a70de1e97c4,22.535519,88.351729,"['9/1, Ram Mohan Dutta Rd', 'Kolkata 700020', 'West Bengal', 'India']",0
8,Fung Fa,52e7cbc2498e9bdd9a1eb232,22.542376,88.392159,"['119/A/1, Matheshwartala Rd', 'Kolkata 700046', 'West Bengal', 'India']",0


### Now we will retrieve data about places for shopping nearby using the Foursquare API and then perform K-Means Clustering and return the suggestions of nearby places to visit.

In [111]:
shopping_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, user_input['latitude'], user_input['longitude'], VERSION, 'shopping', RADIUS, LIMIT)
shopping_url

'https://api.foursquare.com/v2/venues/search?client_id=2C2VEMD5PZD4UOAPK0RYGRBB4XA3PCIMABOKFVMUVO21FEAC&client_secret=C2SK0MD0BWFG3DMDONNMEJ55HEWGIJBRQQWBFED45HDAAWCR&ll=22.55388349859243,88.3516722970781&v=20180604&query=shopping&radius=20000&limit=50'

In [112]:
shopping_results = requests.get(shopping_url).json()
shopping_results

{'meta': {'code': 200, 'requestId': '5ee653eaf89b1820a65b1725'},
 'response': {'venues': [{'id': '56d54c51498e3b3c9ff86414',
    'name': 'The Globe Shopping Mall',
    'location': {'address': '21, Humayun Pl',
     'lat': 22.558963550961092,
     'lng': 88.35243701934814,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.558963550961092,
       'lng': 88.35243701934814}],
     'distance': 570,
     'postalCode': '700087',
     'cc': 'IN',
     'city': 'Kolkata',
     'state': 'West Bengal',
     'country': 'India',
     'formattedAddress': ['21, Humayun Pl',
      'Kolkata 700087',
      'West Bengal',
      'India']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1592152839',
    'hasPerk': False},
   {'id': '5711e3dd498e7

In [113]:
# assign relevant part of JSON to venues
shopping = shopping_results['response']['venues']

# tranform venues into a dataframe
shopping_df = json_normalize(shopping)
shopping_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,56d54c51498e3b3c9ff86414,The Globe Shopping Mall,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592152839,False,"21, Humayun Pl",22.558964,88.352437,"[{'label': 'display', 'lat': 22.55896355096109...",570,700087,IN,Kolkata,West Bengal,India,"[21, Humayun Pl, Kolkata 700087, West Bengal, ...",NaN,NaN
1,5711e3dd498e7d4892cd6082,Shopping,"[{'id': '56aa371be4b08b9a8d573554', 'name': 'E...",v-1592152839,False,"31, Ganesh Chandra Ave",22.568306,88.356557,"[{'label': 'display', 'lat': 22.56830640447288...",1682,700013,IN,Kolkata,West Bengal,India,"[31, Ganesh Chandra Ave, Kolkata 700013, West ...",NaN,NaN
2,4d4faa1a8a592c0f752c8f9f,Vardaan Shopping Market,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592152839,False,"25A, Camac St",22.548193,88.353810,"[{'label': 'display', 'lat': 22.54819321981021...",670,700016,IN,Kolkata,West Bengal,India,"[25A, Camac St, Kolkata 700016, West Bengal, I...",NaN,NaN
3,4fead6a4011c5c131a1e8fa8,Homeland Shopping Mall,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592152839,False,"18B, Ashutosh Mukherjee Rd",22.536282,88.346075,"[{'label': 'display', 'lat': 22.53628245442991...",2042,700020,IN,Kolkata,West Bengal,India,"[18B, Ashutosh Mukherjee Rd, Kolkata 700020, W...",NaN,NaN
4,4db2a089f7b12ee8facdb145,Metro Plaza,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1592152839,False,"1, Ho Chi Minh Sarani",22.547103,88.349063,"[{'label': 'display', 'lat': 22.54710325904829...",801,700071,IN,Kolkata,West Bengal,India,"[1, Ho Chi Minh Sarani, Kolkata 700071, West B...",NaN,NaN


In [114]:
shopping_df = shopping_df[['name', 'id', 'location.lat', 'location.lng', 'location.formattedAddress']]
shopping_df.rename(columns = {'name': 'Name', 'id': 'ID', 'location.lat': 'Latitude', 'location.lng': 'Longitude', 'location.formattedAddress': 'Address'}, inplace = True)

In [115]:
shopping_df.style.set_properties(**{'background-color': 'orange',                                                   
                                    'color': 'black',                       
                                    'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address
0,The Globe Shopping Mall,56d54c51498e3b3c9ff86414,22.558964,88.352437,"['21, Humayun Pl', 'Kolkata 700087', 'West Bengal', 'India']"
1,Shopping,5711e3dd498e7d4892cd6082,22.568306,88.356557,"['31, Ganesh Chandra Ave', 'Kolkata 700013', 'West Bengal', 'India']"
2,Vardaan Shopping Market,4d4faa1a8a592c0f752c8f9f,22.548193,88.353810,"['25A, Camac St', 'Kolkata 700016', 'West Bengal', 'India']"
3,Homeland Shopping Mall,4fead6a4011c5c131a1e8fa8,22.536282,88.346075,"['18B, Ashutosh Mukherjee Rd', 'Kolkata 700020', 'West Bengal', 'India']"
4,Metro Plaza,4db2a089f7b12ee8facdb145,22.547103,88.349063,"['1, Ho Chi Minh Sarani', 'Kolkata 700071', 'West Bengal', 'India']"
5,Dakshinapan Shopping Complex,4bcf06c8cc8cd13a0e33c5cf,22.508749,88.366601,"['2, Gariahat Rd (Near Dhakuria Bridge)', 'Kolkata 700068', 'West Bengal', 'India']"
6,Avishar Shopping Mall,5102aca2e4b077344fd5b9c4,22.504112,88.400030,"['369/4, Eastern Metropolitan Bypass', 'Kolkata 700099', 'West Bengal', 'India']"
7,NBCC Shopping Centre,4dd17a53d164679b8d47d762,22.578099,88.453341,"['CE Block, New Town', 'Kolkata 700156', 'West Bengal', 'India']"
8,Indian-Rain,5b336e373092be002cdeab12,22.593587,88.410501,"['CD-339, Sector-1, Salt Lake', 'Kolkata 700064', 'West Bengal', 'India']"
9,J M Shopping Mall,549d7b30498eafa422480aa5,22.634994,88.219807,['India']


K-Means Clustering with 3 clusters.

In [116]:
shopping_clf = KMeans(n_clusters = 3, init = 'k-means++')
shopping_clf.fit(shopping_df[['Latitude', 'Longitude']])
shopping_labels = shopping_clf.labels_
shopping_centers = shopping_clf.cluster_centers_

In [117]:
shopping_df['Label'] = 0

for i in range(shopping_df.shape[0]):
    shopping_df['Label'][i] = shopping_labels[i]

shopping_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Name,ID,Latitude,Longitude,Address,Label
0,The Globe Shopping Mall,56d54c51498e3b3c9ff86414,22.558964,88.352437,"[21, Humayun Pl, Kolkata 700087, West Bengal, ...",1
1,Shopping,5711e3dd498e7d4892cd6082,22.568306,88.356557,"[31, Ganesh Chandra Ave, Kolkata 700013, West ...",1
2,Vardaan Shopping Market,4d4faa1a8a592c0f752c8f9f,22.548193,88.353810,"[25A, Camac St, Kolkata 700016, West Bengal, I...",1
3,Homeland Shopping Mall,4fead6a4011c5c131a1e8fa8,22.536282,88.346075,"[18B, Ashutosh Mukherjee Rd, Kolkata 700020, W...",1
4,Metro Plaza,4db2a089f7b12ee8facdb145,22.547103,88.349063,"[1, Ho Chi Minh Sarani, Kolkata 700071, West B...",1


Let's visualise the clusters.

In [118]:
# create map
map_clusters = folium.Map(location = [kol_latitude, kol_longitude], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(8)
ys = [i + x + (i*x)**2 for i in range(8)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(shopping_df['Latitude'], shopping_df['Longitude'], shopping_df['Name'], shopping_df['Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

Now, we will find out the nearest cluster.

In [119]:
shopping_centers

array([[22.62439011, 88.41538581],
       [22.53881568, 88.36065322],
       [22.634994  , 88.219807  ]])

Let's find out the respective distances of each cluster center from the hotel.

In [120]:
import math

dist = []

for i in range(3):
    dist.append(math.sqrt((kol_latitude - shopping_centers[i][0])**2 + (kol_longitude - shopping_centers[i][1])**2))
    
dist

[0.09834977603686526, 0.007652238865817854, 0.1636616339895135]

Let's find out the nearest cluster.

In [121]:
idx = dist.index(min(dist))
idx

1

In [122]:
shopping_df[shopping_df['Label'] == idx].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,The Globe Shopping Mall,56d54c51498e3b3c9ff86414,22.558964,88.352437,"['21, Humayun Pl', 'Kolkata 700087', 'West Bengal', 'India']",1
1,Shopping,5711e3dd498e7d4892cd6082,22.568306,88.356557,"['31, Ganesh Chandra Ave', 'Kolkata 700013', 'West Bengal', 'India']",1
2,Vardaan Shopping Market,4d4faa1a8a592c0f752c8f9f,22.548193,88.353810,"['25A, Camac St', 'Kolkata 700016', 'West Bengal', 'India']",1
3,Homeland Shopping Mall,4fead6a4011c5c131a1e8fa8,22.536282,88.346075,"['18B, Ashutosh Mukherjee Rd', 'Kolkata 700020', 'West Bengal', 'India']",1
4,Metro Plaza,4db2a089f7b12ee8facdb145,22.547103,88.349063,"['1, Ho Chi Minh Sarani', 'Kolkata 700071', 'West Bengal', 'India']",1
5,Dakshinapan Shopping Complex,4bcf06c8cc8cd13a0e33c5cf,22.508749,88.366601,"['2, Gariahat Rd (Near Dhakuria Bridge)', 'Kolkata 700068', 'West Bengal', 'India']",1
6,Avishar Shopping Mall,5102aca2e4b077344fd5b9c4,22.504112,88.400030,"['369/4, Eastern Metropolitan Bypass', 'Kolkata 700099', 'West Bengal', 'India']",1


# Final Overview of the project.

#### Input Details of the tourist:

In [125]:
print('Name:        ', user_input['hotel_name'])
print('ID:          ', user_input['hotel_id'])
print('Latitude:    ', user_input['latitude'])
print('Longitude:   ', user_input['longitude'])
print('Address:     ', user_input['Address'])

Name:         The Park Hotel
ID:           4d1348110ad2f04d513fb254
Latitude:     22.55388349859243
Longitude:    88.3516722970781
Address:      ['17, Park St', 'Kolkata 700016', 'West Bengal', 'India']


#### Now, let's respectively print the suggested places for food, coffee, nightlife, fun and shopping.

#### Places for food:

In [130]:
food_df[food_df['Label'] == 5].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Food Bazaar,4fc631bbe4b005fcba5f8871,22.546905,88.353392,"['Pantaloons', 'Kolkata 700016', 'West Bengal', 'India']",5
2,R C S Fast Food,5b0ab58ee1f228002c5797d7,22.541059,88.353649,"['229, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",5
3,Food Court,598eb4d98d0a5310231f7437,22.539206,88.365499,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']",5
9,Aqua Java Fast Food Pvt. Ltd.,4e71c57ce4cd7aa6d6f40a53,22.546865,88.354229,"['10, Wood St', 'Kolkata 700017', 'West Bengal', 'India']",5
13,Aqua Java Fast Food Pvt. Ltd.,4bbe3eda79eaef3b07e4aec3,22.542949,88.355867,"['8/2, Loudon St', 'Kolkata 700016', 'West Bengal', 'India']",5
14,Hunger Food Court,5ca04aaf18d43b002c8c4b59,22.544605,88.354576,"['700017', 'West Bengal', 'India']",5
16,Arsh Fast Food,51dac3e2498e0188907115ec,22.550649,88.362094,['India'],5
17,City Fast Food Center,5916c41f6bd36b0dc9458694,22.551789,88.363664,"['Acharya Jagadish Chandra Bose Rd (Beniapukur Road)', 'Kolkata 700017', 'West Bengal', 'India']",5
19,Al falah fast food,5088fd48e4b0b15cd5f580ad,22.543978,88.359619,"['Kolkata', 'West Bengal', 'India']",5
21,Shimnan Fast Food Centre,50c3145ce4b0e121076eabb6,22.541098,88.353767,"['Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",5


#### Places for coffee:

In [131]:
coffee_df[coffee_df['Label'] == 1].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Cafe Coffee Day,4c6e9eae4ee776b0b4b0581b,22.545478,88.352501,"['4/1, Camac St', 'Kolkata 700016', 'West Bengal', 'India']",1
1,Cafe Coffee Day,4c6ea5834ee776b055e1581b,22.552775,88.352717,"['Park St', 'Kolkata 700017', 'West Bengal', 'India']",1
2,Cafe Coffee Day,5167a710e4b08b494a5adeed,22.555407,88.350313,"['Park St', 'Kolkata 700016', 'West Bengal', 'India']",1
4,The Coffee Bean & Tea Leaf,52c16ed711d2e75296b8c7f6,22.538809,88.365524,"['Quest Mall', 'Kolkata 700017', 'West Bengal', 'India']",1
6,Cafe Coffee Day,4c6eaf425d089c7487bf07c3,22.547381,88.354433,"['Wood St', 'Kolkata 700016', 'West Bengal', 'India']",1
7,Cafe Coffee Day Kolkata Office,50347ce4e4b0bf8f2d057132,22.545424,88.352783,"['Camac St', 'Kolkata 700017', 'West Bengal', 'India']",1
12,Coffee World,4d0e13a8801054816a042296,22.534602,88.364883,"['1, Ballygunge Park Rd', 'Kolkata 700019', 'West Bengal', 'India']",1
13,Cafe Coffee Day,4e5b9da345dd705592b256ab,22.543596,88.358161,"['42/A, Rawdon St', 'Kolkata 700017', 'West Bengal', 'India']",1
17,Cafe Coffee Day,5cf94dea8f90d9002347d428,22.541287,88.349830,"['235/2A, Acharya Jagadish Chandra Bose Rd', 'Kolkata 700020', 'West Bengal', 'India']",1
19,Cafe Coffee Day,4c6ea0814ee776b0f4be581b,22.538542,88.351954,"['Woodburn Pk', 'Kolkata 700020', 'West Bengal', 'India']",1


#### Places for nightlife:

In [132]:
nightlife_df[nightlife_df['Label'] == 0].head(4).style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
5,London Pub,583d8075ea29b85601cdf158,22.547619,88.349700,"['Golden Parkk, Chowringhee', 'India']",0
7,Rishirich Soda Pub,52dd1ccc498e9f6c1d09befe,22.540038,88.361610,"['Beck Bagan Row', 'Kolkata 700017', 'West Bengal', 'India']",0
8,Soda Pub,4fdb69ede4b08187e9dabda9,22.538066,88.345485,"['Sambhu Nath Pandit St', 'Kolkata 700020', 'West Bengal', 'India']",0
9,Rishirich Soda Pub,529e921511d255ec089af691,22.533329,88.347609,"['23A/1/B, Justice Dwarka Nath Rd', 'Kolkata 700020', 'West Bengal', 'India']",0


#### Places for fun:

In [135]:
fun_df[fun_df['Label'] == 0].drop(3).style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,Kolkata Fun Club,5446e0e6498ee9a7ef4d7957,22.543556,88.348875,"['25, Jawaharlal Nehru Rd', 'Kolkata 700016', 'West Bengal', 'India']",0
2,COLORZ - The Funked-Up Store,4ffab423e4b0ae40ee877f91,22.564408,88.354348,"['1/A, Grant St', 'Kolkata 700013', 'West Bengal', 'India']",0
4,Funda & Yener Karakaş,549fc3a0498ef7f11ab5872d,22.532181,88.347939,"['Bbsnsbbaba', 'India']",0
5,Personalized Funda,51c8fb85498e8a70de1e97c4,22.535519,88.351729,"['9/1, Ram Mohan Dutta Rd', 'Kolkata 700020', 'West Bengal', 'India']",0
8,Fung Fa,52e7cbc2498e9bdd9a1eb232,22.542376,88.392159,"['119/A/1, Matheshwartala Rd', 'Kolkata 700046', 'West Bengal', 'India']",0


#### Places for shopping:

In [134]:
shopping_df[shopping_df['Label'] == 1].style.set_properties(**{'background-color': 'orange',                                                   
                                                            'color': 'black',                       
                                                            'border-color': 'white'})

,Name,ID,Latitude,Longitude,Address,Label
0,The Globe Shopping Mall,56d54c51498e3b3c9ff86414,22.558964,88.352437,"['21, Humayun Pl', 'Kolkata 700087', 'West Bengal', 'India']",1
1,Shopping,5711e3dd498e7d4892cd6082,22.568306,88.356557,"['31, Ganesh Chandra Ave', 'Kolkata 700013', 'West Bengal', 'India']",1
2,Vardaan Shopping Market,4d4faa1a8a592c0f752c8f9f,22.548193,88.353810,"['25A, Camac St', 'Kolkata 700016', 'West Bengal', 'India']",1
3,Homeland Shopping Mall,4fead6a4011c5c131a1e8fa8,22.536282,88.346075,"['18B, Ashutosh Mukherjee Rd', 'Kolkata 700020', 'West Bengal', 'India']",1
4,Metro Plaza,4db2a089f7b12ee8facdb145,22.547103,88.349063,"['1, Ho Chi Minh Sarani', 'Kolkata 700071', 'West Bengal', 'India']",1
5,Dakshinapan Shopping Complex,4bcf06c8cc8cd13a0e33c5cf,22.508749,88.366601,"['2, Gariahat Rd (Near Dhakuria Bridge)', 'Kolkata 700068', 'West Bengal', 'India']",1
6,Avishar Shopping Mall,5102aca2e4b077344fd5b9c4,22.504112,88.400030,"['369/4, Eastern Metropolitan Bypass', 'Kolkata 700099', 'West Bengal', 'India']",1
